In [1]:
import os
GPUs = ','.join([str(i) for i in range(0,8)])
num_gpus = len(GPUs.split(','))
os.environ['CUDA_VISIBLE_DEVICES'] = GPUs
os.environ["RAPIDS_NO_INITIALIZE"] = "1" #Vjawa: Added

In [2]:
import dask, dask_cudf
from dask.distributed import Client, wait
from dask_cuda import LocalCUDACluster
import subprocess
import xgboost as xgb

def enable_spilling():
    import cudf
    cudf.set_option("spill", True)
    
def get_ip():
    cmd = "hostname --all-ip-addresses"
    process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE)
    output, error = process.communicate()
    IPADDR = str(output.decode()).split()[0]
    print(IPADDR)
    return IPADDR

def get_cluster():
    ip = get_ip()
    cluster = LocalCUDACluster(ip=ip,)
                               #protocol="ucx", 
                               #rmm_pool_size="31GB")
    client = Client(cluster)
    
    #client.run(enable_spilling)
    return client


In [3]:

client = get_cluster()
client

10.120.104.12


2023-04-13 13:58:45,861 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-04-13 13:58:45,861 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2023-04-13 13:58:45,883 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-04-13 13:58:45,883 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2023-04-13 13:58:45,883 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-04-13 13:58:45,884 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2023-04-13 13:58:45,908 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-04-13 13:58:45,908 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-04-13 13:58:45,908 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2023-04-13 13:58:45,908 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-04-13

Connection method: Cluster object,Cluster type: dask_cuda.LocalCUDACluster
Dashboard: http://10.120.104.12:8787/status,
Dashboard: http://10.120.104.12:8787/status,Workers: 8
Total threads: 8,Total memory: 1.48 TiB
Status: running,Using processes: True
Comm: tcp://10.120.104.12:44353,Workers: 8
Dashboard: http://10.120.104.12:8787/status,Total threads: 8
Started: Just now,Total memory: 1.48 TiB
Comm: tcp://10.120.104.12:41587,Total threads: 1
Dashboard: http://10.120.104.12:45453/status,Memory: 188.82 GiB
Nanny: tcp://10.120.104.12:36599,


In [4]:
%%time

users = dask_cudf.read_parquet('/raid/otto/Otto-Comp/pqs/train_v152_*.pq').persist()#,split_row_groups=True)
users.head()

CPU times: user 1.94 s, sys: 1.32 s, total: 3.27 s
Wall time: 10.5 s


,user,item,pos,wgt,clicks,score_1,score_3,score_4,score_5,score_6,...,cart_repeat9,buy_ratio10,count_item10,unique_item10,hour_mean10,day_mean10,repeat10,buy_ratio_std10,hour_std10,day_std10
0,11098528,11830,0,-1.000,0,-1.0,-1.0,-1.0,-1.0,-1.0,...,1.965084,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,11098528,588923,1,14.500,0,2.0,1.0,-3.0,2.0,4.0,...,2.031830,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,11098528,884502,2,12.485,0,1.0,1.0,-3.0,1.0,2.0,...,1.925403,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,11098528,1182614,3,12.470,0,1.0,1.0,-3.0,1.0,1.5,...,1.904306,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,11098528,571762,4,11.990,0,1.0,1.0,-3.0,1.0,2.0,...,1.953917,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [5]:
%%time
users.shape[0].compute()

CPU times: user 3.04 s, sys: 1.16 s, total: 4.2 s
Wall time: 38.3 s


180125100

In [6]:
%%time
users.memory_usage().sum().compute()/2**30

CPU times: user 285 ms, sys: 45.1 ms, total: 330 ms
Wall time: 421 ms


105.77028216049075

In [7]:
VER = 186
USE = 'clicks'

In [8]:
FEATURES = users.columns[2:]
TARS = [USE]
FEATURES = [f for f in FEATURES if f not in TARS]
print(len(FEATURES))
print( FEATURES)
print(TARS)

152
['pos', 'wgt', 'score_1', 'score_3', 'score_4', 'score_5', 'score_6', 'score_7', 'score_10', 'score_11', 'score_12', 'score_13', 'score_30', 'score_14', 'score_15', 'score_16', 'score_17', 'score_18', 'score_19', 'score_20', 'score_21', 'score_22', 'score_23', 'score_24', 'score_25', 'score_26', 'buy_ratio_5', 'count_item_5', 'count_user_5', 'repeat_5', 'buy_ratio_std_5', 'item_day_std_5', 'user_day_std_5', 'buy_ratio_0_6', 'buy_ratio_1_6', 'buy_ratio_2_6', 'buy_ratio_3_6', 'buy_ratio_4_6', 'buy_ratio_5_6', 'buy_ratio_6_6', 'count_item_0_6', 'count_item_1_6', 'count_item_2_6', 'count_item_3_6', 'count_item_4_6', 'count_item_5_6', 'count_item_6_6', 'count_user_0_6', 'count_user_1_6', 'count_user_2_6', 'count_user_3_6', 'count_user_4_6', 'count_user_5_6', 'count_user_6_6', 'item_day_std_6', 'user_day_std_6', 'buy_ratio', 'count_item', 'count_user', 'hour_mean', 'day_mean', 'repeat', 'buy_ratio_std', 'hour_std', 'day_std', 'buy_ratio3', 'count_item3', 'count_user3', 'hour_mean3', 'day

In [9]:
%%time

dtrain = xgb.dask.DaskQuantileDMatrix(client, users[FEATURES].astype('float32'), users['clicks'].astype('float32'))

CPU times: user 2.85 s, sys: 984 ms, total: 3.84 s
Wall time: 34.3 s


In [10]:
FOLDS = 5
SEED = 42

LR = 0.1

# XGB MODEL PARAMETERS
xgb_parms = { 
    'max_depth':4, 
    'learning_rate':LR, 
    'subsample':0.7,
    'colsample_bytree':0.5, 
    'eval_metric':'map',
    'objective':'binary:logistic',
    'scale_pos_weight':8,
    'tree_method':'gpu_hist',
    'predictor':'gpu_predictor',
    'random_state':SEED
}

In [11]:
%%time

output = xgb.dask.train(
        client,
        xgb_parms,
        dtrain,
        num_boost_round=100,
        evals=[(dtrain, "train")],
    )

/raid/data/mambaforge/envs/rapids-23.04/lib/python3.10/site-packages/xgboost/dask.py:856: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
INFO:distributed.scheduler:Receive client connection: Client-worker-39fcd83a-da3e-11ed-a8db-5cff35c18da6
INFO:distributed.core:Starting established connection to tcp://10.120.104.12:43300
INFO:distributed.scheduler:Receive client connection: Client-worker-39fe71e3-da3e-11ed-a8d9-5cff35c18da6
INFO:distributed.core:Starting established connection to tcp://10.120.104.12:43304
INFO:distributed.scheduler:Receive client connection: Client-worker-3b06fb40-da3e-11ed-a8df-5cff35c18da6
INFO:distributed.core:Starting established connection to tcp://10.120.104.12:43308
INFO:distributed.scheduler:Receive client connection: Client-worker-3b140185-da3e-11ed-a8d3-5cff35c18da6
INFO:distributed.core:Starting established connection to tcp://10.120.104.12:43312
INFO:distributed.scheduler:Receive client connecti

[0]	train-map:0.16610
[1]	train-map:0.17588
[2]	train-map:0.17653
[3]	train-map:0.17719
[4]	train-map:0.18376
[5]	train-map:0.18461
[6]	train-map:0.18543
[7]	train-map:0.18540
[8]	train-map:0.18551
[9]	train-map:0.18577
[10]	train-map:0.18714
[11]	train-map:0.18710
[12]	train-map:0.18707
[13]	train-map:0.18699
[14]	train-map:0.18730
[15]	train-map:0.18802
[16]	train-map:0.18812
[17]	train-map:0.18834
[18]	train-map:0.18877
[19]	train-map:0.18902
[20]	train-map:0.18930
[21]	train-map:0.18945
[22]	train-map:0.18995
[23]	train-map:0.18997
[24]	train-map:0.19000
[25]	train-map:0.19006
[26]	train-map:0.19008
[27]	train-map:0.19108
[28]	train-map:0.19177
[29]	train-map:0.19214
[30]	train-map:0.19229
[31]	train-map:0.19275
[32]	train-map:0.19338
[33]	train-map:0.19319
[34]	train-map:0.19408
[35]	train-map:0.19459
[36]	train-map:0.19486
[37]	train-map:0.19586
[38]	train-map:0.19633
[39]	train-map:0.19655
[40]	train-map:0.19664
[41]	train-map:0.19696
[42]	train-map:0.19705
[43]	train-map:0.1973

# TODO: implement group kfold in dask cudf